In [19]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_ollama import ChatOllama
from langchain_huggingface import HuggingFaceEmbeddings  # HuggingFace嵌入

load_dotenv(dotenv_path=r"E:\Workplace\GitHub\Learning\LangChain\.env", override=True)

model = ChatOpenAI(
                    # temperature=0.1,
                    # max_tokens=4096,
                    # presence_penalty=1.2,
                    # model='deepseek-chat',
                    # openai_api_key=os.getenv('DEEPSEEK_API_KEY'),
                    # openai_api_base="https://api.deepseek.com/v1",
                    # model='gpt-4o-mini',
                    # openai_api_key=os.getenv('CLOSEAI_API_KEY'),
                    # openai_api_base="https://api.openai-proxy.org/v1",
                    # model='yi-lightning',
                    # openai_api_key=os.getenv('YI_API_KEY'),
                    # openai_api_base="https://api.lingyiwanwu.com/v1",
                    model='ernie-4.0-turbo-8k-latest',
                    openai_api_key=os.getenv('BD_API_KEY'),
                    openai_api_base="https://qianfan.baidubce.com/v2",
                    default_headers={"appid": "app-dz4Hp7OK"}
                )

# model = ChatOllama(
#                     temperature=0,
#                     num_predict=4096,
#                     repeat_penalty=1.2,
#                     model="llama3.2:3b"
#                 )

embedding = HuggingFaceEmbeddings(
            model_name=r"E:/Workplace/GitHub/Embedding/maidalun/bce-embedding-base_v1",
            model_kwargs={"device": "cuda",
                          "trust_remote_code": True},
            encode_kwargs={"normalize_embeddings": True},
        )

# End-to-end agent

In [2]:
# Import relevant functionality
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent

# Create the agent
memory = MemorySaver()
search = TavilySearchResults(max_results=2)
tools = [search]
agent_executor = create_react_agent(model, tools, checkpointer=memory)

# Use the agent
config = {"configurable": {"thread_id": "abc123"}}
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="hi im bob! and i live in sf")]}, config
):
    print(chunk)
    print("----")

for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather where I live?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='Hi Bob! 很高兴认识你，旧金山是个很棒的地方！我能帮你什么忙吗？你对旧金山的天气、新闻、活动或者其他什么主题感兴趣吗？告诉我，我尽量帮你找相关信息！', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 41, 'prompt_tokens': 9, 'total_tokens': 50, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'ernie-4.0-turbo-8k-latest', 'system_fingerprint': None, 'finish_reason': 'normal', 'logprobs': None}, id='run-14ff1082-a160-48af-9bfb-c4e4d0cc6513-0', usage_metadata={'input_tokens': 9, 'output_tokens': 41, 'total_tokens': 50, 'input_token_details': {}, 'output_token_details': {}})]}}
----
{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': '1a0d58c8b168d000', 'function': {'arguments': '{"query": "旧金山当前天气"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 57, 'total_tokens': 74, 'completion_tokens_details': N

# Define tools

In [3]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults(max_results=2)
search_results = search.invoke("what is the weather in SF")
print(search_results)
# If we want, we can create other tools.
# Once we have all the tools we want, we can put them in a list that we will reference later.
tools = [search]

[{'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.775, 'lon': -122.4183, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1736407088, 'localtime': '2025-01-08 23:18'}, 'current': {'last_updated_epoch': 1736406900, 'last_updated': '2025-01-08 23:15', 'temp_c': 10.4, 'temp_f': 50.7, 'is_day': 0, 'condition': {'text': 'Clear', 'icon': '//cdn.weatherapi.com/weather/64x64/night/113.png', 'code': 1000}, 'wind_mph': 4.0, 'wind_kph': 6.5, 'wind_degree': 25, 'wind_dir': 'NNE', 'pressure_mb': 1023.0, 'pressure_in': 30.2, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 50, 'cloud': 0, 'feelslike_c': 9.8, 'feelslike_f': 49.7, 'windchill_c': 12.4, 'windchill_f': 54.3, 'heatindex_c': 12.2, 'heatindex_f': 53.9, 'dewpoint_c': 10.3, 'dewpoint_f': 50.6, 'vis_km': 16.0, 'vis_miles': 9.0, 'uv': 0.0, 'gust_mph': 7.9, 'gust_kph': 12.6}}"}, {'url': 'https://www.meteoprog.com/weather/Sanfrancisc

# Using Language Models

In [4]:
from langchain_core.messages import HumanMessage

response = model.invoke([HumanMessage(content="hi!")])
response.content

"Hello! It's great to hear from you. How can I assist you today? Whether you have a question, need information, or just want to chat, I'm here to help."

In [5]:
model_with_tools = model.bind_tools(tools)

In [6]:
response = model_with_tools.invoke([HumanMessage(content="Hi!")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: Hello！很高兴听到你的声音。今天我能为你做点什么吗？如果你有任何问题或需要帮助，请随时告诉我！
ToolCalls: []


In [7]:
response = model_with_tools.invoke([HumanMessage(content="What's the weather in SF?")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: 
ToolCalls: [{'name': 'tavily_search_results_json', 'args': {'query': '旧金山今天天气'}, 'id': '1a0d58db79e97000', 'type': 'tool_call'}]


# Create the agent

In [20]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model, tools)

# Run the agent

In [15]:
response = agent_executor.invoke({"messages": [HumanMessage(content="hi!")]})

response["messages"]

[HumanMessage(content='hi!', additional_kwargs={}, response_metadata={}, id='11d8a494-351b-48a2-b287-672e84bdf273'),
 AIMessage(content='Hi! 很高兴见到你。请问有什么我可以帮助你的吗？', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 2, 'total_tokens': 15, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'ernie-4.0-8k-latest', 'system_fingerprint': None, 'finish_reason': 'normal', 'logprobs': None}, id='run-d75be2cc-596b-4871-8737-1ca63a22b64f-0', usage_metadata={'input_tokens': 2, 'output_tokens': 13, 'total_tokens': 15, 'input_token_details': {}, 'output_token_details': {}})]

In [16]:
response = agent_executor.invoke(
    {"messages": [HumanMessage(content="whats the weather in sf?")]}
)
response["messages"]

[HumanMessage(content='whats the weather in sf?', additional_kwargs={}, response_metadata={}, id='7cbfbf74-9d00-4c6b-b0f2-a1e137bb56af'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': '1a0d59a0812ba000', 'function': {'arguments': '{"query": "旧金山天气"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 6, 'total_tokens': 22, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'ernie-4.0-8k-latest', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-db037a4b-1679-4fb4-b36a-1e2b90961bf0-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': '旧金山天气'}, 'id': '1a0d59a0812ba000', 'type': 'tool_call'}], usage_metadata={'input_tokens': 6, 'output_tokens': 16, 'total_tokens': 22, 'input_token_details': {}, 'output_token_details': {}}),
 ToolMessage(content='[{"url": "https://www.meteoprog.com/c

# Streaming Messages

In [17]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather in beijing？")]}
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': '1a0d59a6062a0000', 'function': {'arguments': '{"query": "北京今天天气"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 7, 'total_tokens': 24, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'ernie-4.0-8k-latest', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-dfd45b66-7d1f-460c-bc9c-665996cc97cd-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': '北京今天天气'}, 'id': '1a0d59a6062a0000', 'type': 'tool_call'}], usage_metadata={'input_tokens': 7, 'output_tokens': 17, 'total_tokens': 24, 'input_token_details': {}, 'output_token_details': {}})]}}
----
{'tools': {'messages': [ToolMessage(content='[{"url": "https://forecast.weather.com.cn/town/weathern/101010100.shtml", "content": "å\x8f°é£\x8eè·¯å¾\x84 ç©ºé\x97´å¤©

# Streaming tokens

In [18]:
async for event in agent_executor.astream_events(
    {"messages": [HumanMessage(content="whats the weather in sf?")]}, version="v1"
):
    kind = event["event"]
    if kind == "on_chain_start":
        if (
            event["name"] == "Agent"
        ):  # Was assigned when creating the agent with `.with_config({"run_name": "Agent"})`
            print(
                f"Starting agent: {event['name']} with input: {event['data'].get('input')}"
            )
    elif kind == "on_chain_end":
        if (
            event["name"] == "Agent"
        ):  # Was assigned when creating the agent with `.with_config({"run_name": "Agent"})`
            print()
            print("--")
            print(
                f"Done agent: {event['name']} with output: {event['data'].get('output')['output']}"
            )
    if kind == "on_chat_model_stream":
        content = event["data"]["chunk"].content
        if content:
            # Empty content in the context of OpenAI means
            # that the model is asking for a tool to be invoked.
            # So we only print non-empty content
            print(content, end="|")
    elif kind == "on_tool_start":
        print("--")
        print(
            f"Starting tool: {event['name']} with inputs: {event['data'].get('input')}"
        )
    elif kind == "on_tool_end":
        print(f"Done tool: {event['name']}")
        print(f"Tool output was: {event['data'].get('output')}")
        print("--")

--
Starting tool: tavily_search_results_json with inputs: {'query': '旧金山天气'}
Done tool: tavily_search_results_json
Tool output was: content='[{"url": "https://www.meteoprog.com/cn/weather/Sanfrancisco/", "content": "旧金山 旧金山 旧金山 美国, 加利福尼亚 旧金山 天气 目前在 旧金山 08:10 PM - 18 十一月, 星期一 +10° +11° +11° +14° +12° 感觉像 +9°C 凉爽的 天气固定在 05:31 PM © Meteoprog.com 旧金山天气: 阴, 无降水 旧金山天气: 不定云况, 无降水 旧金山天气: 多云间晴, 无降水 旧金山天气: 阴, 无明显降水 旧金山天气: 多云, 无降水 旧金山天气: 阴, 无降水 旧金山天气: 多云, 无降水 旧金山天气: 阴, 无降水 旧金山天气: 多云, 无降水 旧金山天气: 阴, 无降水 旧金山天气: 不定云况, 无降水 旧金山 : 详细信息 37° 46′ 12″ 北 -123° 35′ 24″ 西 黄昏 民事 黄昏 天文 黄昏 航海 在过去 75 年的观察中，今天在 旧金山 的最高气温为 +22,1°C，记录于 2005。 旧金山 的最大降雨量（38,39 毫米）记录在 1982。 1965的最大阵风高达 19,59 米/秒。 美国 +11° +14° +15° +20° +11° +14° +11° +17° +14° +10° +17° +9° +17° +11° +19° +17° +13° +16° +14° 旧金山 +10° 旧金山 今天的天气 旧金山 美国 © Meteoprog.com 2003-2024"}, {"url": "https://weather.com/zh-CN/weather/tenday/l/54f9d8baac32496f6b5497b4bf7a277c3e2e6cc5625de69680e6169e7e38e9a8", "content": "。\\n周五 09 | 夜间\\n少云。 最低 6°C。 北 风 10 到 15 每 km 

# Adding in memory

In [21]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

agent_executor = create_react_agent(model, tools, checkpointer=memory)

config = {"configurable": {"thread_id": "abc123"}}

In [22]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="hi im bob!")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='Hi Bob! Nice to meet you. How can I assist you today, or do you have any specific questions?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 4, 'total_tokens': 27, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'ernie-4.0-turbo-8k-latest', 'system_fingerprint': None, 'finish_reason': 'normal', 'logprobs': None}, id='run-857554a3-5bb3-416c-a037-4ef7710bb166-0', usage_metadata={'input_tokens': 4, 'output_tokens': 23, 'total_tokens': 27, 'input_token_details': {}, 'output_token_details': {}})]}}
----


In [23]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats my name?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='Hello Bob! Your name is Bob. Is there anything else I can help you with today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 31, 'total_tokens': 50, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'ernie-4.0-turbo-8k-latest', 'system_fingerprint': None, 'finish_reason': 'normal', 'logprobs': None}, id='run-9fc245b8-3f15-44f9-bad1-c5936ee91c5b-0', usage_metadata={'input_tokens': 31, 'output_tokens': 19, 'total_tokens': 50, 'input_token_details': {}, 'output_token_details': {}})]}}
----


In [24]:
config = {"configurable": {"thread_id": "xyz123"}}
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats my name?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content="Hello! I'm sorry, but I don't have the ability to directly know or retrieve your name. If you need help remembering or finding out your name, I suggest looking at personal identification documents or asking someone who knows you.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 48, 'prompt_tokens': 4, 'total_tokens': 52, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'ernie-4.0-turbo-8k-latest', 'system_fingerprint': None, 'finish_reason': 'normal', 'logprobs': None}, id='run-afac70fc-29d8-4658-93f9-3602e644e412-0', usage_metadata={'input_tokens': 4, 'output_tokens': 48, 'total_tokens': 52, 'input_token_details': {}, 'output_token_details': {}})]}}
----
